In [10]:
import os
import sys

dir = os.path.abspath('')
parent = os.path.dirname(dir)
if not parent in sys.path:
    sys.path.append(parent)


In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.ops import box_iou
from torchvision.transforms import Compose, Resize, ToTensor

from data.custom_celeba import CelebA, xywh2xyxy
from models.my_net import MyNet


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')


Using device cuda


In [13]:
transform = Compose([
    ToTensor(),
    Resize((256, 256))
])

target_transform = Compose([
    xywh2xyxy,
    torch.Tensor.float
])

training_data = CelebA(
    root='../CelebA',
    split='train',
    transform=transform,
    target_transform=target_transform,
    resize=True
)

test_data = CelebA(
    root='../CelebA',
    split='test',
    transform=transform,
    target_transform=target_transform,
    resize=True
)


In [14]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=6)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=6)


In [15]:
model = MyNet().to(device)


In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * dataloader.batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    return loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (box_iou(pred, y).diagonal() > 0.4).sum().item()
    test_loss /= num_batches
    correct /= size

    print(
        f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')
    
    return correct, test_loss


In [17]:
epochs = 18
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

_, test_loss = test(test_dataloader, model, loss_fn)

correct = 0
test_losses = [test_loss]
train_losses = [test_loss]

for epoch in range(1, epochs+1):
    print(f'Epoch {epoch}\n-------------------------------')
    new_train_loss = train(train_dataloader, model, loss_fn, optimizer)
    new_correct, new_test_loss = test(test_dataloader, model, loss_fn)

    train_losses.append(new_train_loss)
    test_losses.append(new_test_loss)

    if new_correct > correct:
        torch.save(model.state_dict(), 'model.pth')
        print('Accuracy increased. Saved PyTorch Model State to model.pth')
        correct = new_correct

print('Done!')


Test Error: 
 Accuracy: 0.0%, Avg loss: 17849.574444 

Epoch 1
-------------------------------
loss: 17732.816406  [    0/162770]
loss: 16286.016602  [ 6400/162770]
loss: 9295.501953  [12800/162770]
loss: 4601.637695  [19200/162770]
loss: 1317.222412  [25600/162770]
loss: 257.202759  [32000/162770]
loss: 243.105911  [38400/162770]
loss: 236.788147  [44800/162770]
loss: 161.580246  [51200/162770]
loss: 140.994156  [57600/162770]
loss: 105.087067  [64000/162770]
loss: 141.307602  [70400/162770]
loss: 166.127045  [76800/162770]
loss: 111.314346  [83200/162770]
loss: 119.032059  [89600/162770]
loss: 99.780029  [96000/162770]
loss: 127.679718  [102400/162770]
loss: 126.846481  [108800/162770]
loss: 115.400024  [115200/162770]
loss: 134.314514  [121600/162770]
loss: 87.054886  [128000/162770]
loss: 56.862026  [134400/162770]
loss: 80.918388  [140800/162770]
loss: 90.772705  [147200/162770]
loss: 70.622116  [153600/162770]
loss: 69.606262  [160000/162770]
Test Error: 
 Accuracy: 97.9%, Avg lo